In [33]:
#!pip install geopandas
#!pip install geopy
#!pip install pgeocode
#!pip install seaborn

In [57]:

import setup
%matplotlib inline


In [59]:

# setup.file_pre_processing()

In [30]:
crime_data = pd.read_csv('crime_data_post_processing.csv')
hospitals = pd.read_csv('hospitals_data_post_processing.csv')


In [108]:
crime_data.head()

,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,...,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY,lat_long
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,...,1778,3609,4995,13791,3543,464,318416,29,510,"(29.6993748, -91.20677)"
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,...,165,662,1482,1753,189,28,49746,5,35,"(35.2505701, -90.3192299)"
2,"Alexander County, IL",1664.700485,3,1,4,604,8040,8040,2,2,...,5,119,82,184,12,2,7629,17,3,"(37.1801529, -89.3502834)"
3,"Kenedy County, TX",1456.310680,4,1,4,2681,444,444,1,1,...,1,2,5,4,4,0,412,48,261,"(27.221984, -97.79136)"
4,"De Soto Parish, LA",1447.402430,5,1,4,1137,26971,26971,3,3,...,17,368,149,494,60,0,27083,22,31,"(31.901498, -93.670989)"


In [109]:
hospitals.head()

,Provider ID,Hospital Name,Address,City,State,ZIP Code,Hospital Type,Hospital Ownership,Emergency Services,Hospital overall rating,Mortality national comparison,Patient experience national comparison,Effectiveness of care national comparison,full_address,lat_long
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Acute Care Hospitals,Government - Hospital District or Authority,Yes,3,Same as the national average,Below the national average,Same as the national average,1108 ROSS CLARK CIRCLE DOTHAN AL 36301,"(31.216402000000002, -85.36564064472296)"
1,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,Acute Care Hospitals,Government - Hospital District or Authority,Yes,3,Below the national average,Same as the national average,Same as the national average,2505 U S HIGHWAY 431 NORTH BOAZ AL 35957,None
2,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,Acute Care Hospitals,Government - Hospital District or Authority,Yes,2,Below the national average,Below the national average,Same as the national average,205 MARENGO STREET FLORENCE AL 35631,"(34.7946245, -87.68303582998244)"
3,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,Acute Care Hospitals,Voluntary non-profit - Private,Yes,2,Same as the national average,Same as the national average,Below the national average,702 N MAIN ST OPP AL 36467,"(31.2928787, -86.25490030241093)"
4,10008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,Acute Care Hospitals,Proprietary,Yes,3,Same as the national average,Not Available,Same as the national average,101 HOSPITAL CIRCLE LUVERNE AL 36049,"(31.6937493, -86.26470160380265)"


In [113]:
print(len(hospitals))

4812


In [122]:
print(len(crime_data))

3136


In [115]:
#drop null lat long hosp. and crime
hospitals = hospitals[hospitals['lat_long'].notna()]
print(len(hospitals))



3928


In [123]:
crime_data = crime_data[crime_data['lat_long'] !="(Na,Na)"]
print(len(crime_data))

3130


In [19]:
import pgeocode
nomi = pgeocode.Nominatim('US')

# Haversine Distance - finding closest hospital 

In [124]:
#Inputs: str of state
#Output: df of hospitals + (lat,long) for that state
def return_matching_state_df(state):
    hospitals_for_state = hospitals[hospitals['State']==state]
    hospitals_for_state=hospitals_for_state[['Hospital Name','lat_long']]
    return hospitals_for_state

#Inputs: series frome crime data
#Output: str of Hospital Name from hospital dataset 
def find_closest_hospital(series):
    lat_long = series['lat_long']
    state = series['county_name'].split(', ')[1]
    df = return_matching_state_df(state)
    lat_long_list = df.lat_long.tolist()
    #https://readthedocs.org/projects/pgeo
    distances = pgeocode.haversine_distance([lat_long], lat_long_list)
    return min(distances)

In [125]:
crime_data['closest_hospital'] = crime_data.apply(find_closest_hospital, axis=1)

<ipython-input-125-2d38cd2b4309>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_data['closest_hospital'] = crime_data.apply(find_closest_hospital, axis=1)


In [126]:
crime_data

,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,...,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY,lat_long,closest_hospital
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,...,3609,4995,13791,3543,464,318416,29,510,"(29.6993748, -91.20677)",739.736789
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,...,662,1482,1753,189,28,49746,5,35,"(35.2505701, -90.3192299)",42.570008
2,"Alexander County, IL",1664.700485,3,1,4,604,8040,8040,2,2,...,119,82,184,12,2,7629,17,3,"(37.1801529, -89.3502834)",32.956523
3,"Kenedy County, TX",1456.310680,4,1,4,2681,444,444,1,1,...,2,5,4,4,0,412,48,261,"(27.221984, -97.79136)",65.156318
4,"De Soto Parish, LA",1447.402430,5,1,4,1137,26971,26971,3,3,...,368,149,494,60,0,27083,22,31,"(31.901498, -93.670989)",15.321499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3131,"Ohio County, IN",0.000000,3132,1,4,762,6084,0,2,1,...,0,2,2,0,0,5994,18,115,"(38.9480496, -84.9581728)",19.243752
3132,"Newton County, MS",0.000000,3133,1,4,1465,21545,3346,3,1,...,0,4,0,1,0,21689,28,101,"(35.896255, -93.160938)",303.524371
3133,"Jerauld County, SD",0.000000,3134,1,4,2424,2108,2108,1,1,...,0,1,3,1,0,2066,46,73,"(44.0797601, -98.6182042)",4.354322
3134,"Cimarron County, OK",0.000000,3135,1,4,2167,2502,2502,2,2,...,0,1,2,0,0,2335,40,25,"(36.7475509, -102.5402407)",2.862427


# group by zipcode to find avg hospital rating 

In [96]:
hospitals['ZIP Code'].value_counts()

77030    6
70508    5
76104    5
73120    4
19104    4
        ..
75401    1
23505    1
41017    1
14642    1
68845    1
Name: ZIP Code, Length: 4416, dtype: int64

In [ ]:
crime_data['zip_code'].value_counts()

In [ ]:
crime_rate_per_100000_by_zip = crime_data.groupby('zip_code')['crime_rate_per_100000']

# Make scatterplot with  zip codes on X axis and crime_rate_per_100000 on Y axis 

In [ ]:
#x axis will be zip codes ranging from lowest zip code possible(00501) 
#to highest zip code possible (99950)

In [ ]:
titlefont = {'size': 16, 'fontname':'Comic Sans MS'}

sns.lineplot(x="zip codes", y="crime rate per 100000", data=)


In [ ]:
# Make barchart with 3 farthest/closest hopsitals to county vs. crime_rate_per_100000 on Y axis 

In [ ]:
three_closest_hospitals = 

## Problem Statement
My goal with this notebook is to use to answer the question: *is class positively correlated with survival on the ship Titanic?*

## Data

To answer this question I will use a dataset from Kaggle called titanic 
(link here: [titanic dataset](https://www.kaggle.com/c/titanic)). I collected this by downloading the csv. The data is stored in a csv format. There are 891 rows and 12 columns. The total size of the file is 59 kB. 

## Data cleaning and manipulation

To address my central question I did the following data cleaning steps:
    ....
    
This are all contained in the file data_cleaning_functions.py. 

## Analysis

To present my analysis I first consider a table of survival rates for each class. Here I can see that the proportion of passengers in the first class who survived is much higher than for the second and third classes. 

However, it is possible that this gap is not trustworthy if there is very little data in one of these classes. So I will also plot survival and include the size of the category in the plot. 

In the plot although we can see there is more data in the third class the amount of data in the first class is not so small that I am concerned. I do think there is a statistically valid relationship between class and survival. However, to be triple sure, I'll check the correlation coefficient between plcass and survival rate and check that it is statistically significant. 

Surprisingly, although the correlation coefficient is very strong negative, it is not statistically signifanct (probably because I only have three values and have lost all information about the size of the data). 


Now I see a weaker but much more significant relationship between the two variables. However, it the pearson correlation coefficient isn't really set up for this kind of comparison, so in future work I will use a better measure. 

## Take-aways
For now, I am convinced that as class goes up from 3 -> 1 there is an increase in survival rates. While the relationship is strong and seems to be significant, I will test the significant levels more thoroughly in future work.  

## References 

Include all references to sources for the code, sources for any packages or methods you used and a guide to all of the files you wrote and are including. 